# MapReduce

## Data

Скачать файл VisitsSteram.tsv https://www.kaggle.com/c/avito-context-ad-clicks 

В VisitsStream.tsv лежит информация про пользователей, которые открывают сайт. Используя классический MapReduce необходимо посчитать топ 10 пользователей с самыми длинными по времени сессиями и время этой самой длинной сессии (в секундах).

Сессия определяется следующим образом - это окно времени, внутри которого временное расстояние от двух соседних посещений не более 15 минут.

Иными словами - если пользователь зашел на сайт в момент X и последнее предыдущее посещение сайта в момент Y было не позднее чем 15 минут назад, то сессия "продлевается" до текущего момента. Если же временное расстояние от X до Y более 15 минут, то считается, что предыдущая сессия закончилась в момент Y, а новая сессия началась в момент X.

Выводить нужно только уникальных пользователей и для каждого такого пользователя находить время самой длинной его сессии.

In [1]:
! cat data/VisitsStream.tsv | head 

59703	1259356	469877	2015-04-25 00:00:00.0
154389	1846749	27252551	2015-04-25 00:00:00.0
218628	2108380	31685325	2015-04-25 00:00:00.0
231535	837110	18827716	2015-04-25 00:00:00.0
282306	1654210	29363673	2015-04-25 00:00:00.0
295068	601505	588324	2015-04-25 00:00:00.0
501897	158476	4103261	2015-04-25 00:00:00.0
655394	631692	9860544	2015-04-25 00:00:00.0
765603	804403	29475627	2015-04-25 00:00:00.0
790289	121085	23309988	2015-04-25 00:00:00.0
cat: write error: Broken pipe


In [2]:
# delete header
# ! for filename in data/*; do sed $filename -i -e '1d' && echo finish $filename ; done

In [3]:
! cat data/VisitsStream.tsv | head -n 200000 > test.tsv

cat: write error: Broken pipe


### user [sessions]

In [4]:
%%writefile user_date.py

import sys

def mapper():
    for line in sys.stdin:
        user, *_, date = [i.strip() for i in line.split('\t')]
        date = date[:-2]
        print(f'{user}\t{date}')

def reducer():
    user, date = next(sys.stdin).split('\t')
    date = [date.strip()]
    for line in sys.stdin:
        user_current, date_current = line.split('\t')
        date_current = date_current.strip()
        if  user != user_current:
            print(f'{user}\t{date}')
            user, date = user_current, [date_current]
        else:
            date += [date_current]
    print(f'{user}\t{date}')

if __name__ == '__main__':
    func = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[func]()


Writing user_date.py


In [5]:
! cat test.tsv | \
    python user_date.py map | \
    sort -n -k 1 | \
    python user_date.py reduce > sessions.txt

### top 10 longest session

In [7]:
%%writefile user_session.py

import sys
from datetime import datetime, timedelta

def mapper():
    interval = timedelta(minutes=15)
    for line in sys.stdin:
        user, date = [i.strip() for i in line.split('\t')]
        date = iter(eval(date))
        date_max = timedelta(seconds=0)
        date_start = datetime.strptime(next(date), '%Y-%m-%d %H:%M:%S')
        date_end = date_start

        for i in date:
            date_next = datetime.strptime(i, '%Y-%m-%d %H:%M:%S')
            if (date_next - date_end) > interval:
                session_long = date_next - date_start
                if session_long > date_max:
                    date_max = session_long
                date_start, date_end = date_end, date_end
            else:
                date_end = date_next

        session_long = date_next - date_start
        if session_long > date_max:
            date_max = session_long

        date_max = int(date_max.total_seconds())
        print(f'{user}\t{date_max}')


# def reducer():
#     user, count = next(sys.stdin).split('\t')
#     count = int(count.strip())
#     for line in sys.stdin:
#         user_current, count_current = line.split('\t')
#         count_current = int(count_current.strip())
#         if user != user_current:
#             print(f'{user}\t{count}')
#             user, count = user_current, count_current
#         else:
#             count += count_current
#     print(f'{user} {count}')


if __name__ == '__main__':
    func = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[func]()


Writing user_session.py


In [12]:
! cat session.txt | \
    python user_session.py map | \
    sort -k2nr > top10.txt

In [13]:
! cat top10.txt | head -n 10

4285777	4329
1203081	4323
3327027	4321
2657836	4318
4077325	4314
2845015	4311
296473	4311
4321061	4310
457844	4310
499117	4309
cat: write error: Broken pipe
